In [1]:
## Background:

## Risksmart and its contemporary GRACE capture event related information among other data points such as controls, change initiatives etc. This unexplored repository provides a rich source of data that augments (and often precedes) remediation related information.​
## Remediation Analytics and Remediation Portfolio Management are working closely to best utilise Risksmart/GRACE data in order to:​
## Build an automated capability to flag remediation related events, creating efficiencies and reducing risk.​
## Help with sequencing and prioritisation of remediation events.

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine import url
import sqlalchemy
import pandas as pd
import numpy as np
import xlwings as xw
import sys
import os
import io
import glob
import os, sys
import win32com.client
import shutil
from zipfile import ZipFile
import time
import warnings
import xlwings as xw
warnings.filterwarnings('ignore')

# %%timeit
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 2000)
pd.options.display.max_colwidth = 2000

In [2]:
path = 'C:\\Users\\XXXX'
file = 'Risksmart data 2014 to current.xlsx'

# load data into dataframe
df = pd.read_excel(path + file, sheetname= 'Sheet2', index_col=None)

df1 = df[['EVENT_ID','TITLE','DESCRIPTION','NFI_REMEDIATION_RATING']].drop_duplicates()
df1 = df1.reset_index()

In [ ]:
import nltk
from nltk.tokenize import TreebankWordTokenizer
from collections import Counter
import string
nltk.download('stopwords')

def bag_of_words(sentence):
    # remove punctuation marks
    translator = str.maketrans('', '', string.punctuation)
    sentence = sentence.translate(translator)
    
    # remove numeric:
    sentence = ''.join([i for i in sentence if not i.isdigit()]) 
    
    # tokenize the words in event description
    tokenizer = TreebankWordTokenizer()
    tokens = tokenizer.tokenize(sentence.lower())

    # remove stop words
    stopwords = nltk.corpus.stopwords.words('english')
    tokens2 = [x for x in tokens if x not in stopwords]
    bag_of_words = Counter(tokens2)
    return dict(bag_of_words)

In [6]:
df1['NFI_REMEDIATION_RATING'].value_counts()

Insignificant    12931
Minor             2608
Moderate           519
Major               35
Critical             5
Name: NFI_REMEDIATION_RATING, dtype: int64

In [42]:
## tokenization - occurrence of words and number of frequency
import nltk
from nltk.tokenize import TreebankWordTokenizer
from collections import Counter
import string

df_freq=pd.DataFrame()

for i in range(0,len(df2)):
    bag = bag_of_words(df2.loc[i].DESCRIPTION)
    bag.update( {'EVENT_ID' : df2.loc[i].EVENT_ID})
    bag.update( {'NFI_REMEDIATION_RATING' : df2.loc[i].NFI_REMEDIATION_RATING})
    df_freq = df_freq.append([bag])

## specifying filters - key identifiers 
filters = {'remediation':not None, 'refund':not None, 'impact':not None}
pd.Series(filters)

df_freq[list(filters)] == pd.Series(filters)
(df_freq[list(filters)] == pd.Series(filters)).any(axis=1)

In [ ]:
filtered=pd.DataFrame()
filtered = filtered.append(df_freq.loc[(df_freq[list(filters)] == pd.Series(filters)).any(axis=1)])

In [128]:
filtered[['EVENT_ID','NFI_REMEDIATION_RATING']]

,EVENT_ID,NFI_REMEDIATION_RATING
0,35802911,Critical
0,79945708,Critical


In [ ]:
## training the predicted model using Random forest


import random
import nltk
from nltk.corpus import twitter_samples
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
df5 = df5.fillna(0)

def casewhen(row):
    if row['NFI_REMEDIATION_RATING'] == 'Critical':
        return 1
    elif row['NFI_REMEDIATION_RATING'] =='Major':
        return 2
    elif row['NFI_REMEDIATION_RATING'] =='Moderate':
        return 3
    elif row['NFI_REMEDIATION_RATING'] =='Minor':
        return 4
    elif row['NFI_REMEDIATION_RATING'] =='Insignificant':
        return 5
    else:
        return 6

df5['RATING'] = df5.apply(casewhen, axis=1)


# Create a list of the feature column's names
features = df5.columns[1:-1]

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(df5[features] , df5['RATING'] ,random_state = 1)

df5['is_train'] = np.random.uniform(0, 1, len(df5)) <= .75
train, test = df5[df5['is_train']==True], df5[df5['is_train']==False]

In [ ]:

X_train = train[features]

X_test = test[features]

# y_train = pd.factorize(train['NFI_REMEDIATION_RATING'])[0]

y_train = np.array(train['RATING'])

# y_test = pd.factorize(test['NFI_REMEDIATION_RATING'])[0]

y_test = np.array(test['RATING'])


rf = RandomForestClassifier(n_estimators=100,n_jobs=4,random_state=10)
rf.fit(X_train,y_train)

preds = rf.predict(X_test)
print(accuracy_score(y_test,preds))

In [ ]:
pd.crosstab(y_test, preds, rownames=['Actual'], colnames=['Predicted'])